<a href="https://colab.research.google.com/github/Achronaz/cs4186cp/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#enable GPU
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi

Wed Apr 21 10:14:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install gdown
#download darknet
%cd /content/
!rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet.git
%cd /content/darknet/
# download yolov4.conv.137
!gdown https://drive.google.com/uc?id=1g2vbXo6rB9G2IqZFBnD-BzqwTcTyL_aq

# build darknet
!sed -i s/GPU=0/GPU=1/g Makefile
!sed -i s/CUDNN=0/CUDNN=1/g Makefile
!sed -i s/CUDNN_HALF=0/CUDNN_HALF=1/g Makefile
!sed -i s/OPENCV=0/OPENCV=1/g Makefile
!sed -i s/LIBSO=0/LIBSO=1/g Makefile
!make

#download dataset
%cd /content/darknet/data/
!gdown https://drive.google.com/u/0/uc?id=15H9e9jxeSlQ2FIygwrdF-jCRezTWsy83
!unzip dataset.zip
%cd /content/darknet/
!cp data/custom/custom5.data custom5.data
!cp data/custom/custom5.cfg custom5.cfg

串流輸出內容已截斷至最後 5000 行。
  inflating: custom/hamburger/285384.jpg  
  inflating: custom/hamburger/285384.txt  
  inflating: custom/hamburger/286323.jpg  
  inflating: custom/hamburger/286323.txt  
  inflating: custom/hamburger/2863754.jpg  
  inflating: custom/hamburger/2863754.txt  
  inflating: custom/hamburger/2874704.jpg  
  inflating: custom/hamburger/2874704.txt  
  inflating: custom/hamburger/2875062.jpg  
  inflating: custom/hamburger/2875062.txt  
  inflating: custom/hamburger/2877643.jpg  
  inflating: custom/hamburger/2877643.txt  
  inflating: custom/hamburger/2879938.jpg  
  inflating: custom/hamburger/2879938.txt  
  inflating: custom/hamburger/2882999.jpg  
  inflating: custom/hamburger/2882999.txt  
  inflating: custom/hamburger/2884546.jpg  
  inflating: custom/hamburger/2884546.txt  
  inflating: custom/hamburger/2891622.jpg  
  inflating: custom/hamburger/2891622.txt  
  inflating: custom/hamburger/2894838.jpg  
  inflating: custom/hamburger/2894838.txt  
  inflating: cu

In [3]:
%cd /content/darknet
!cp cfg/yolov4-custom.cfg custom5.cfg
!sed -i s/batch=64/batch=64/g custom5.cfg
#lower subdivisions to speed up training process (more gpu memory required)
!sed -i s/subdivisions=16/subdivisions=32/g custom5.cfg
!sed -i s/max_batches\ =\ 500500/max_batches\ =\ 10000/g custom5.cfg
!sed -i s/steps=400000,450000/steps=8000,9000/g custom5.cfg
!sed -i s/filters=255/filters=30/g custom5.cfg
!sed -i s/classes=80/classes=5/g custom5.cfg

with open('/content/darknet/custom5.data', 'w') as f:
  f.write("classes = 5\n")
  f.write("train = data/custom/train.txt\n")
  f.write("valid = data/custom/test.txt\n")
  f.write("names = data/custom/custom5.names\n")
  f.write('backup = backup/')

/content/darknet


In [4]:
%cd /content/darknet

# train from scratch
!./darknet detector train custom5.data custom5.cfg yolov4.conv.137 -map -dont_show
# train from checkpoint
#!./darknet detector train custom5.data custom5.cfg backup/custom5_last.weights -map -dont_show

/content/darknet
 CUDA-version: 11000 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    608 x 608 x   3 ->  608 x 608 x  32 0.639 BF
   1 conv     64       3 x 3/ 2    608 x 608 x  32 ->  304 x 304 x  64 3.407 BF
   2 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   3 route  1 		                           ->  304 x 304 x  64 
   4 conv     64       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  64 0.757 BF
   5 conv     32       1 x 1/ 1    304 x 304 x  64 ->  304 x 304 x  32 0.379 BF
   6 conv     64       3 x 3/ 1    304 x 304 x  32 ->  304 x 304 x  64 3.407 BF
   7 Shortcut Laye

In [ ]:
# evaluate mAP
!./darknet detector map custom5.data custom5.cfg backup/custom5_last.weights
# calculate Recall Precision
!./darknet detector recall custom5.data custom5.cfg backup/custom5_last.weights